Insert your desired values in the sections marked with '###' to use the code.

In [ ]:
for i in range(10, 60, 10) :
    downsampling_rates = i

    epochs = ###

    window_size = ###
    train_step_size = ###
    test_step_size = 10

    import pandas as pd
    import numpy as np
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import Dataset, DataLoader
    from torch.utils.data.dataset import random_split
    from pytorchtools import EarlyStopping
    from sklearn.metrics import roc_auc_score
    import pickle
    import time
    import matplotlib.pyplot as plt
    import random

    seed = ###
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    class DownsampledDataset(Dataset):
        def __init__(self, data, downsampling_rates):
            self.data = data
            self.downsampling_rates = downsampling_rates

        def __len__(self):
            return len(self.data)

        def __getitem__(self, index):
            x = self.data[index]
            samples = []
            labels = []
            for rate in range(self.downsampling_rates * 2):
                downsampled_x = self.downsample(x, rate+1)
                padded_x = self.pad_zeros(downsampled_x, len(x[0]))
                samples.append(padded_x)
                label = [0] * (self.downsampling_rates * 2)
                label[rate] = 1
                labels.append(label)
            samples = torch.tensor(samples, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.float32)
            return samples, labels

        def downsample(self, x, rate):
            a = None
            if rate <= self.downsampling_rates :
                a = [x[i][::rate] for i in range(len(x))]
            else :
                rate = rate - self.downsampling_rates
                a = []
                for i in x :
                    b = []
                    for j in range(0, len(i), rate*2) :
                        b += i[j:j+rate]
                    a.append(b)
            return a

        def pad_zeros(self, x, target_length):
            if len(x[0]) < target_length:
                num_zeros = target_length - len(x[0])
                x = [sample + [0] * num_zeros for sample in x]
            return x

    class NEW1DCNN(nn.Module):
        def __init__(self, downsampling_rates, input_dim):
            super(NEW1DCNN, self).__init__()
            self.downsampling_rates = downsampling_rates * 2
            self.convs = nn.Sequential(nn.Conv1d(input_dim, 16, kernel_size=3),
                                       nn.ReLU(),
                                       nn.MaxPool1d(kernel_size=2, stride=2),
                                       nn.Conv1d(16, 32, kernel_size=3),
                                       nn.ReLU(),
                                       nn.MaxPool1d(kernel_size=2, stride=2),
                                       nn.Flatten())
            self.fc = None
            self.softmax = nn.Softmax(dim=1)

        def forward(self, x):
            results = []
            for i in range(self.downsampling_rates):
                output = self.convs(x[:,i,:,:])
                if self.fc is None:
                    self.fc = nn.Linear(output.size(-1), self.downsampling_rates).to(output.device)
                output = self.fc(output)
                output = self.softmax(output)
                results.append(output)
            results = torch.stack(results, dim=0)
            results = torch.transpose(results, 0, 1).reshape(-1, results.shape[-1])
            return results

    def train_model(data, downsampling_rates, batch_size, input_dim, epochs):
        dataset = DownsampledDataset(data, downsampling_rates)
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = NEW1DCNN(downsampling_rates, input_dim=input_dim).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=###, weight_decay=###)

        saving_model_path = './saving_model/'
        early_stopping = EarlyStopping(saving_model_path=saving_model_path, patience=###, verbose=True)

        for epoch in range(epochs):
            train_loss = 0.0
            val_loss = 0.0

            model.train()
            for batch_inputs, batch_labels in train_dataloader:
                batch_inputs = batch_inputs.to(device)
                batch_labels = batch_labels.to(device)
                optimizer.zero_grad()
                outputs = model(batch_inputs)
                labels = batch_labels.view(-1, batch_labels.shape[-1])

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            train_loss /= len(train_dataloader)
            torch.save(model.state_dict(), '%s_epoch_%04d.pt' % (saving_model_path, epoch))

            model.eval()
            with torch.no_grad():
                for batch_inputs, batch_labels in val_dataloader:
                    batch_inputs = batch_inputs.to(device)
                    batch_labels = batch_labels.to(device)                
                    outputs = model(batch_inputs)
                    labels = batch_labels.view(-1, batch_labels.shape[-1])

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                val_loss /= len(val_dataloader)

            print(f"Epoch {epoch+1}/{epochs}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

            early_stopping(val_loss, model)        
            if early_stopping.early_stop:
                print("Early stopping")
                break

        model.load_state_dict(torch.load('%s_checkpoint.pt' % (saving_model_path)))

        return model

    def test_model(data, downsampling_rates, model):
        start_time = time.time()
        dataset = DownsampledDataset(data, downsampling_rates)
        test_dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
        criterion = nn.CrossEntropyLoss()

        scores = []

        for test_input, test_label in test_dataloader:
            test_input = test_input.to(device)
            test_label = test_label.to(device)
            outputs = model(test_input)
            labels = test_label.view(-1, test_label.shape[-1])

            anomaly_score = criterion(outputs, labels)
            scores.append(anomaly_score.cpu().detach().numpy())

        scaled_scores = min_max_scale(scores)
        auroc = roc_auc_score(test_target, scaled_scores)
        print('auroc:', auroc)

        end_time = time.time()
        execution_time = end_time - start_time
        print(f"test: {execution_time} sec")
        return scaled_scores

    def min_max_scale(scores):
        min_val = min(scores)
        max_val = max(scores)
        scaled_scores = [(x - min_val) / (max_val - min_val) for x in scores]
        return scaled_scores

    def create_segments_and_labels(dataframe, window_size, step_size):
        segments = []
        target = []
        for start_pos in range(0, len(dataframe), step_size):
            end_pos = start_pos + window_size
            if end_pos > len(dataframe):
                break
            segment = dataframe.iloc[start_pos:end_pos].values
            segments.append(segment[:,:-1])
            if 1 in segment[:,-1]:
                target.append(1)
            else:
                target.append(0)

        segments = np.array(segments).transpose((0, 2, 1))
        segments = segments.tolist()

        return segments, target

    with open('###', 'rb') as f:
        traindata = pickle.load(f)
    traindata = pd.DataFrame(traindata)
    traindata[traindata.columns[-1]] = traindata[traindata.columns[-1]].astype(int)

    with open('###', 'rb') as f:
        testdata = pickle.load(f)
    testdata = pd.DataFrame(testdata)
    testdata[testdata.columns[-1]] = testdata[testdata.columns[-1]].astype(int)

    print(traindata[traindata.columns[-1]].unique())
    print(testdata[testdata.columns[-1]].unique())

    train_data, train_target = create_segments_and_labels(traindata, window_size, train_step_size)
    test_data, test_target = create_segments_and_labels(testdata, window_size, test_step_size)
    train_data = [x for x, y in zip(train_data, train_target) if y == 0]
    print('train data :', len(train_data))
    print('train target :', len(train_target))
    print('test data :', len(test_data))
    print('test target :', len(test_target))

    batch_size = ###
    input_dim = len(train_data[0])

    start_time = time.time()
    model = train_model(train_data, downsampling_rates, batch_size, input_dim, epochs)
    end_time = time.time()
    execution_time = end_time - start_time

    print(f"train: {execution_time} sec")

    scaled_scores = test_model(test_data, downsampling_rates, model)

    %reset -f